In [2]:
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.backend as K
from keras.metrics import MeanIoU
import os
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image, ImageOps
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Dropout, UpSampling2D, concatenate, Activation, multiply, Reshape,BatchNormalization
from keras.layers import GlobalAveragePooling2D, Reshape
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler,EarlyStopping

2024-12-24 14:18:02.004753: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-24 14:18:02.004884: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-24 14:18:02.156494: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def load_data(image_filenames, mask_filenames, image_dir, mask_dir, image_size=(512, 512)):
    image_list, mask_list = [], []
    for img_file, mask_file in zip(image_filenames, mask_filenames):
        if img_file.endswith(".png") and mask_file.endswith(".png"):
            image = cv2.imread(os.path.join(image_dir, img_file), cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, image_size)
            image = image / 255.0  # Normalize to [0, 1]
            image = np.expand_dims(image, axis=-1)  # Add channel dimension
            image_list.append(image)
            
            mask = cv2.imread(os.path.join(mask_dir, mask_file), cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, image_size)
            mask = mask / 255.0  # Normalize to [0, 1]
            mask = np.expand_dims(mask, axis=-1)  # Add channel dimension
            mask_list.append(mask)
    return np.array(image_list), np.array(mask_list)


In [4]:
# Define paths to your image and mask directories
image_dir = '/kaggle/input/fives-a-fundus-image/FIVES A Fundus Image Dataset for AI-based Vessel Segmentation/train/Original/'
mask_dir = '/kaggle/input/fives-a-fundus-image/FIVES A Fundus Image Dataset for AI-based Vessel Segmentation/train/Ground truth/'

# Get list of image and mask filenames
image_filenames = os.listdir(image_dir)
mask_filenames = os.listdir(mask_dir)

# Sort filenames to ensure corresponding images and masks match
image_filenames.sort()
mask_filenames.sort()
# Load images and masks
images, masks = load_data(image_filenames, mask_filenames, image_dir, mask_dir)


In [5]:
# Define paths to your image and mask directories
image_dir = '/kaggle/input/fives-a-fundus-image/FIVES A Fundus Image Dataset for AI-based Vessel Segmentation/test/Original/'
mask_dir = '/kaggle/input/fives-a-fundus-image/FIVES A Fundus Image Dataset for AI-based Vessel Segmentation/test/Ground truth/'

# Get list of image and mask filenames
image_filenames = os.listdir(image_dir)
mask_filenames = os.listdir(mask_dir)

# Sort filenames to ensure corresponding images and masks match
image_filenames.sort()
mask_filenames.sort()
# Load images and masks
Images_test, Masks_test = load_data(image_filenames, mask_filenames, image_dir, mask_dir)

In [6]:
images =np.array(images)
masks =np.array(masks)
Images_test =np.array(Images_test)
Masks_test =np.array(Masks_test)

images_val, images_test, masks_val, masks_test = train_test_split(Images_test, Masks_test, test_size=0.5, random_state=42)

# Print shapes to verify
print("Training images shape:", images.shape)
print("Training masks shape:", masks.shape)
print("test images shape:", images_test.shape)
print("test masks shape:", masks_test.shape)
print("validation images shape:", images_val.shape)
print("validation masks shape:", masks_val.shape)
# Ensure your images and masks are 4D tensors with consistent shapes

images = tf.expand_dims(images, axis=-1)  # If your images are (batch, height, width)
masks = tf.expand_dims(masks, axis=-1)    # If your masks are (batch, height, width)

images_test = tf.expand_dims(images_test, axis=-1)
masks_test = tf.expand_dims(masks_test, axis=-1)

images_val = tf.expand_dims(images_val, axis=-1)
masks_val = tf.expand_dims(masks_val, axis=-1)

Training images shape: (600, 512, 512, 1)
Training masks shape: (600, 512, 512, 1)
test images shape: (100, 512, 512, 1)
test masks shape: (100, 512, 512, 1)
validation images shape: (100, 512, 512, 1)
validation masks shape: (100, 512, 512, 1)


In [7]:
def dice_loss(y_true, y_pred):
    smooth = 1.0  # To avoid division by zero
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

# Define PSNR metric
def psnr(y_true, y_pred):
    # Ensure y_true and y_pred have the same shape
    y_true = tf.ensure_shape(y_true, y_pred.shape)
    return tf.image.psnr(y_true, y_pred, max_val=1.0)

stop_callback=EarlyStopping(monitor='val_accuracy',restore_best_weights=True,verbose=1 , mode='max',patience=50,start_from_epoch=0)

# Custom SSIM metric
def ssim_metric(y_true, y_pred):
    return tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, Conv2DTranspose, BatchNormalization, Activation, Add
from tensorflow.keras.models import Model

In [9]:
def resunet(input_shape=(512, 512, 1)):
    def res_block(x, filters):
        shortcut = x
        x = tf.keras.layers.Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
        x = tf.keras.layers.Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
        shortcut = tf.keras.layers.Conv2D(filters, (1, 1), padding='same')(shortcut)
        x = tf.keras.layers.Add()([x, shortcut])
        return x

    inputs = tf.keras.Input(input_shape)
    c1 = res_block(inputs, 64)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)
    c2 = res_block(p1, 128)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
    bottleneck = res_block(p2, 256)
    u1 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(bottleneck)
    u1 = tf.keras.layers.concatenate([u1, c2])
    u1 = res_block(u1, 128)
    u2 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(u1)
    u2 = tf.keras.layers.concatenate([u2, c1])
    u2 = res_block(u2, 64)
    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(u2)
    model = tf.keras.Model(inputs, outputs)
    return model


In [10]:
resunet = resunet((512,512,1))

In [11]:
print("Images shape:", images.shape)  # Should be (batch_size, 512, 512, 1)
print("Masks shape:", masks.shape)    # Should be (batch_size, 512, 512, 1)


Images shape: (600, 512, 512, 1, 1)
Masks shape: (600, 512, 512, 1, 1)


In [12]:
# Ensure tensors have the correct shape by removing all unnecessary dimensions first
images = tf.squeeze(images)  # Removes all dimensions of size 1
masks = tf.squeeze(masks)
images_val = tf.squeeze(images_val)
masks_val = tf.squeeze(masks_val)
images_test = tf.squeeze(images_test)
masks_test = tf.squeeze(masks_test)

# Add back the single channel dimension to ensure (batch_size, 512, 512, 1)
images = tf.expand_dims(images, axis=-1)
masks = tf.expand_dims(masks, axis=-1)
images_val = tf.expand_dims(images_val, axis=-1)
masks_val = tf.expand_dims(masks_val, axis=-1)
images_test = tf.expand_dims(images_test, axis=-1)
masks_test = tf.expand_dims(masks_test, axis=-1)

# Verify the new shapes
print("Images shape:", images.shape)  # Should be (batch_size, 512, 512, 1)
print("Masks shape:", masks.shape)    # Should be (batch_size, 512, 512, 1)


Images shape: (600, 512, 512, 1)
Masks shape: (600, 512, 512, 1)


In [13]:
resunet.compile(optimizer='adam', loss='binary_crossentropy', 
              metrics=['accuracy', tf.keras.metrics.Precision(), MeanIoU(num_classes=2),dice_loss,ssim_metric,psnr])

In [14]:
history = resunet.fit(images,masks,validation_data=(images_val,masks_val), epochs=50, batch_size=1, verbose = 1)

Epoch 1/50


2024-12-24 14:21:16.697365: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 262144: 1.24674, expected 0.706489
2024-12-24 14:21:16.697436: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 262145: 2.29621, expected 1.75596
2024-12-24 14:21:16.697453: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 262146: 2.24082, expected 1.70057
2024-12-24 14:21:16.697469: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 262147: 1.8087, expected 1.26845
2024-12-24 14:21:16.697486: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 262148: 2.13502, expected 1.59477
2024-12-24 14:21:16.697498: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 262149: 2.59674, expected 2.05649
2024-12-24 14:21:16.697510: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 262150: 2.16053, expected 1.62028
2024-12-24 14:21:16.

  2/600 ━━━━━━━━━━━━━━━━━━━━ 47s 79ms/step - accuracy: 0.8799 - dice_loss: 0.8619 - loss: 0.6008 - mean_io_u: 0.4600 - precision: 0.0015 - psnr: 7.0224 - ssim_metric: 0.0034   

I0000 00:00:1735050092.509700     108 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


600/600 ━━━━━━━━━━━━━━━━━━━━ 72s 82ms/step - accuracy: 0.8123 - dice_loss: 0.9572 - loss: 2093674528768.0000 - mean_io_u: 0.3140 - precision: 0.0724 - psnr: 9.2356 - ssim_metric: 0.4071 - val_accuracy: 0.5049 - val_dice_loss: 0.8219 - val_loss: 8045072893345792.0000 - val_mean_io_u: 0.2872 - val_precision: 0.1140 - val_psnr: 3.8086 - val_ssim_metric: 0.3719
Epoch 2/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 46s 76ms/step - accuracy: 0.8433 - dice_loss: nan - loss: nan - mean_io_u: 0.3457 - precision: 0.0940 - psnr: nan - ssim_metric: nan - val_accuracy: 0.9237 - val_dice_loss: nan - val_loss: nan - val_mean_io_u: 0.4551 - val_precision: 0.0000e+00 - val_psnr: nan - val_ssim_metric: nan
Epoch 3/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 46s 76ms/step - accuracy: 0.9186 - dice_loss: nan - loss: nan - mean_io_u: 0.3855 - precision: 0.0000e+00 - psnr: nan - ssim_metric: nan - val_accuracy: 0.9237 - val_dice_loss: nan - val_loss: nan - val_mean_io_u: 0.4551 - val_precision: 0.0000e+00 - val_psnr: nan - val_ssim_


KeyboardInterrupt



In [17]:
resunet.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy', MeanIoU(num_classes=2)])  # Replace 'mean_io_u' with your actual IoU metric


In [20]:
test_loss, test_accuracy, test_miou = resunet.evaluate(images_test, masks_test, verbose=1)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Mean IoU: {test_miou}")

UnknownError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code

  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 701, in start

  File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/opt/conda/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 534, in dispatch_que

In [ ]:
resunet.save("resunet_segmentation_model.h5")  


In [21]:
resunet.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 512, 512,  │        640 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 512, 512,  │     36,928 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 512, 512,  │        128 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 512, 512,  │          0 │ conv2d_1[0][0],   │
│                     │ 64)               │            │ conv2d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 256, 256,  │          0 │ add[0][0]         │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 256, 256,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 256, 256,  │    147,584 │ conv2d_3[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 256, 256,  │      8,320 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 256, 256,  │          0 │ conv2d_4[0][0],   │
│                     │ 128)              │            │ conv2d_5[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 128, 128,  │          0 │ add_1[0][0]       │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 128, 128,  │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 128, 128,  │    590,080 │ conv2d_6[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 128, 128,  │     33,024 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 128, 128,  │          0 │ conv2d_7[0][0],   │
│                     │ 256)              │            │ conv2d_8[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 256, 256,  │    131,200 │ add_2[0][0]       │
│ (Conv2DTranspose)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256, 256,  │          0 │ conv2d_transpose

 Total params: 1,944,321 (7.42 MB)

 Trainable params: 1,944,321 (7.42 MB)

 Non-trainable params: 0 (0.00 B)